## Advection-Diffusion


#### Problem Setup

\begin{equation} 
0 = au_x + bu_y + cu_{xx} + du_{yy} - u_t \; \; \; \; \; \; \; \; \; (*)
\end{equation}

For the generation of our training data we use the same method as in PDE-Net in Tensorflow (for details please refer to that chapter in the paper) with:

$a = b = 2$ <br> $c = d = 0.5$ <br>

$u: \mathbb{R}^3 \rightarrow \mathbb{R}$ is some solution to $(*)$. <br>
$f: \mathbb{R}^3 \rightarrow \mathbb{R}, \;f(x,y,t) = 0$ <br>
$X_i := (x_i, y_i, t_i) \in [0,1] \times [0,1] \times [0, 0.135] \subseteq \mathbb{R}^3$ for $i \in \{1, \dotsc, n\}$

and our known function values will be $\{u(X_i), f(X_i)\}_{i \in \{1, \dotsc, n\}}$.

We assume that $u$ can be represented as a Gaussian process with SE kernel.

$u \sim \mathcal{GP}(0, k_{uu}(X_i, X_j; \theta))$, where $\theta = \{\sigma, l_x, l_y, l_t\}$.

Set the linear operator to:

$\mathcal{L}_X^{\phi} := a\partial_x + b\partial_y + c\partial_{x,x} + d\partial_{y,y} - \partial_t$

so that

$\mathcal{L}_X^{\phi} u = f = 0$

Problem at hand: Estimate $\phi:=\{a,b,c,d\}$.


#### Step 1: Simulate data

In [112]:
import time
import numpy as np
import sympy as sp
import numpy.fft as fft
from scipy.linalg import solve_triangular
import scipy.optimize as opt

In [113]:
# Global variables: x, y, t, n, y_u, y_f, s

# Number of data samples
n = 110

# Noise of our data:
s = 1e-7

In [114]:
options = {'mesh_size': [250, 250],     # How large is the (regular) 2D-grid of function values for a fixed t.
                                        # Keep mesh_size[0] = mesh_size[1]
           'layers': 9,                 # Layers of the NN. Also counting the initial layer!
           'dt': 0.0015,                 # Time discretization. We step dt*(layers - 1) forward in time.
           'batch_size': 1,             # We take a batch of sub-grids in space
           'noise_level': 0.0,          # Can add some noise to the data (not taken 1 to 1, gets multiplied by stddev)
           'downsample_by': 1,          # Size of sub-grids (in space) * downsample_by = mesh_size
           }

In [115]:
def initgen(mesh_size, freq=3, boundary='Periodic'):
    """
    Returns function values for t=0 on a regular grid of size 'mesh_size' in [0, 2*pi]x[0, 2*pi] as a matrix
    """
    # Default: (mesh_size, freq, boundary) = ([250, 250], 4, 'Periodic')
    if np.iterable(freq):
        return freq
    # 250x250 normally distributed variables IFFTed and FFTed:
    x = _initgen_periodic(mesh_size, freq=freq)
    x = x * 100
    if boundary.upper() == 'DIRICHLET':
        dim = x.ndim
        for i in range(dim):
            y = np.arange(mesh_size[i]) / mesh_size[i]
            y = y * (1 - y)
            s = np.ones(dim, dtype=np.int32)
            s[i] = mesh_size[i]
            y = np.reshape(y, s)
            x = x * y
        x = x[[slice(1, None), ] * dim]
        x = x * 16
    return x

In [116]:
def _initgen_periodic(mesh_size, freq=3):
    # np.random.seed(50)
    # Default: (mesh_size, freq) = ([250, 250], 4)
    dim = len(mesh_size)
    # Default: 250x250-matrix of normally distributed variables
    x = np.random.randn(*mesh_size)
    coe = fft.ifftn(x)
    # set frequency of generated initial value
    # Array of random ints in [freq, 2*freq - 1]
    freqs = np.random.randint(freq, 2 * freq, size=[dim, ])
    # freqs = [10,10]
    for i in range(dim):
        perm = np.arange(dim, dtype=np.int32)
        perm[i] = 0
        perm[0] = i
        # Permutes for i = 1 and does nothing for i = 0.
        coe = coe.transpose(*perm)
        coe[freqs[i] + 1:-freqs[i]] = 0
        coe = coe.transpose(*perm)
    x = fft.fftn(coe)
    assert np.linalg.norm(x.imag) < 1e-8
    x = x.real
    return x

In [117]:
def pad_input_2(input, pad_by):
    """
    We increase the size of input for all j by pad_by on each side of the matrix
    by inserting values from the opposite side
    """
    mesh_size = input.shape[0]

    B = np.eye(mesh_size, dtype=np.float32)
    for i in range(pad_by):
        a = np.zeros(mesh_size, dtype=np.float32)
        a[mesh_size - i - 1] = 1
        B = np.concatenate(([a], B), axis=0)
    for i in range(pad_by):
        a = np.zeros(mesh_size, dtype=np.float32)
        a[i] = 1
        B = np.concatenate((B, [a]), axis=0)

    return B @ input @ B.T

def downsample(sample, scale):
    """
    Returns a regular somewhat random sub-grid of sample, whose size is reduced by a factor of 'scale'.
    """
    # np.random.seed(50)
    idx1 = slice(np.random.randint(scale), None, scale)
    idx2 = slice(np.random.randint(scale), None, scale)
    # idx1 = slice(1, None, scale)
    # idx2 = slice(0, None, scale)

    for kwarg in sample:
        sample[kwarg] = sample[kwarg][idx1, idx2]
    return sample

def addNoise(sample, noise, layers):
    # Adding noise to u0
    mean = sample['u0'].mean()
    stdvar = np.sqrt(((sample['u0'] - mean) ** 2).mean())
    size = sample['u0'].shape
    startnoise = np.random.standard_normal(size)
    sample['u0'] = sample['u0'] + noise * stdvar * startnoise

    # Adding noise to ut, t > 0
    for l in range(1, layers):
        arg = 'u' + str(l)
        size = sample[arg].shape
        endnoise = np.random.standard_normal(size)
        sample[arg] = sample[arg] + noise * stdvar * endnoise

    return sample

In [118]:
def generate(options):
    """
    Generating data / function-values on a regular grid of space-time, adding noise and taking a batch of
    down-sampled regular sub-grids of this grid. This batch will contain the samples to train our network with.

    :param options: The dictionary of user-specified options (cf. main.py). Contains e.g. the grid-dimensions and noise
    :return: A batch (as a list) of samples (as dictionaries), that in turn consist of (noisy) function values on
             down-sampled sub-grids for all dt-layers.
    """
    # u_t = a*u_x + b*u_y + c*u_{xx} + d*u_{yy}

    # Variable declarations
    nx = options['mesh_size'][0]
    ny = options['mesh_size'][1]
    nt = options['layers']
    batch_size = options['batch_size']
    dt = options['dt']
    noise_level = options['noise_level']
    downsample_by = options['downsample_by']

    a = 2
    b = 2
    c = 0.5
    d = 0.5

    dx = 2 * np.pi / (nx - 1)
    dy = 2 * np.pi / (ny - 1)

    ## Needed for plotting:
    # x = np.linspace(0, 2*np.pi, num = nx)
    # y = np.linspace(0, 2*np.pi, num = ny)
    # X, Y = np.meshgrid(x, y)

    batch = []

    for i in range(batch_size):
        ############ Change the following lines to implement your own data ############

        # Assign initial function:
        u = initgen(options['mesh_size'], freq=4, boundary='Periodic')

        ## Plotting the initial function:
        # fig = plt.figure(figsize=(11,7), dpi=100)
        # ax = fig.gca(projection='3d')
        # surf = ax.plot_surface(X, Y, u[:], cmap=cm.viridis)
        #
        # plt.show()

        sample = {}
        sample['u0'] = u

        for n in range(nt - 1):
            un = pad_input_2(u, 2)[1:, 1:]

            u = (un[1:-1, 1:-1] + c * dt / dx ** 2 * (un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, 0:-2])
                 + d * dt / dy ** 2 * (un[2:, 1: -1] - 2 * un[1:-1, 1:-1] + un[0:-2, 1:-1])
                 + a * dt / dx * (un[1:-1, 2:] - un[1:-1, 1:-1])
                 + b * dt / dy * (un[2:, 1:-1] - un[1:-1, 1:-1]))[:-1, :-1]

            sample['u' + str(n + 1)] = u

        ## sample should at this point be a dictionary with entries 'u0', ..., 'uL', where L = nt                   ##
        ## For a given j, sample['uj'] is a matrix of size nx x ny containing the function values at time-step dt*j ##
        ##############################################################################################################

        downsample(sample, downsample_by)
        addNoise(sample, noise_level, nt)

        batch.append(sample)

    return batch

Note, that we only take $n$ data samples with their corresponding function values from the grid.

In [119]:
def simulate_data():
    batch = generate(options)
    
    x_val_arr = []
    y_val_arr = []
    t_val_arr = []
    val_arr = []
    
    for i in range(n): 
        t_rand = np.random.randint(options['layers'])
        # Data should be in [0,1]
        x_rand = np.random.randint(options['mesh_size'][0]//(2*np.pi))
        y_rand = np.random.randint(options['mesh_size'][1]//(2*np.pi))

        val = batch[0]['u' + str(t_rand)][x_rand, y_rand]

        x_val = 2 * np.pi / (options['mesh_size'][0] - 1) * x_rand
        y_val = 2 * np.pi / (options['mesh_size'][1] - 1) * y_rand
        t_val = t_rand * options['dt']
        
        x_val_arr.append(x_val)
        y_val_arr.append(y_val)
        t_val_arr.append(t_val)
        val_arr.append(val)
        
    return (np.array(x_val_arr), np.array(y_val_arr), np.array(t_val_arr), np.array(val_arr), np.zeros(len(val_arr)))
(x,y,t,y_u,y_f) = simulate_data()

#### Step 2: Evaluate kernels

$k_{uu}(X_i, X_j; \theta) = \sigma exp(-\frac{1}{2l_x}(x_i-x_j)^2 - \frac{1}{2l_y}(y_i-y_j)^2 - \frac{1}{2l_t}(t_i-t_j)^2)$

In [120]:
x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, a, b, c, d = sp.symbols('x_i x_j y_i y_j t_i t_j sigma l_x l_y l_t a b c d')
kuu_sym = sigma*sp.exp(-1/(2*l_x)*((x_i - x_j)**2) - 1/(2*l_y)*((y_i - y_j)**2) - 1/(2*l_t)*((t_i - t_j)**2))
kuu_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t), kuu_sym, "numpy")
def kuu(x, y, t, sigma, l_x, l_y, l_t):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kuu_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t)
    return k

$k_{ff}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} \mathcal{L}_{X_j}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= a^2\partial_{x_i, x_j}k_{uu} + ab \partial_{x_i, y_j}k_{uu} + ac \partial_{x_i, x_j, x_j}k_{uu} + ad \partial_{x_i, y_j, y_j}k_{uu} - a \partial_{x_i, t_j}k_{uu} 
\\+ ba\partial_{y_i, x_j}k_{uu} + b^2\partial_{y_i, y_j}k_{uu} + bc\partial_{y_i, x_j, x_j}k_{uu} + bd\partial_{y_i, y_j, y_j}k_{uu} - b\partial_{y_i, t_j}k_{uu}
\\+ ca\partial_{x_i, x_i, x_j}k_{uu}+ cb\partial_{x_i, x_i, y_j}k_{uu}+ c^2\partial_{x_i, x_i, x_j, x_j}k_{uu}+ cd\partial_{x_i, x_i, y_j, y_j}k_{uu}- c\partial_{x_i, x_i, t_j}k_{uu}
\\+ da\partial_{y_i, y_i,x_j}k_{uu}+ db\partial_{y_i, y_i, y_j}k_{uu}+ dc\partial_{y_i, y_i, x_j, x_j}k_{uu}+ d^2\partial_{y_i, y_i, y_j, y_j}k_{uu}- d\partial_{y_i, y_i, t_j}k_{uu}
\\- a\partial_{t_i, x_j}k_{uu}- b\partial_{t_i, y_j}k_{uu}- c\partial_{t_i, x_j, x_j}k_{uu}- d\partial_{t_i, y_j, y_j}k_{uu}+ \partial_{t_i, t_j}k_{uu}$

In [121]:
kff_sym = a**2*sp.diff(kuu_sym, x_i, x_j) \
        + a*b*sp.diff(kuu_sym, x_i, y_j) \
        + a*c*sp.diff(kuu_sym, x_i, x_j, x_j) \
        + a*d*sp.diff(kuu_sym, x_i, y_j, y_j) \
        - a*sp.diff(kuu_sym, x_i, t_j) \
        + b*a*sp.diff(kuu_sym, y_i, x_j) \
        + b**2*sp.diff(kuu_sym, y_i, y_j) \
        + b*c*sp.diff(kuu_sym, y_i, x_j, x_j) \
        + b*d*sp.diff(kuu_sym, y_i, y_j, y_j) \
        - b*sp.diff(kuu_sym, y_i, t_j) \
        + c*a*sp.diff(kuu_sym, x_i, x_i, x_j) \
        + c*b*sp.diff(kuu_sym, x_i, x_i, y_j) \
        + c**2*sp.diff(kuu_sym, x_i, x_i, x_j, x_j) \
        + c*d*sp.diff(kuu_sym, x_i, x_i, y_j, y_j) \
        - c*sp.diff(kuu_sym, x_i, x_i, t_j) \
        + d*a*sp.diff(kuu_sym, y_i, y_i, x_j) \
        + d*b*sp.diff(kuu_sym, y_i, y_i, y_j) \
        + d*c*sp.diff(kuu_sym, y_i, y_i, x_j, x_j) \
        + d**2*sp.diff(kuu_sym, y_i, y_i, y_j, y_j) \
        - d*sp.diff(kuu_sym, y_i, y_i, t_j) \
        - a*sp.diff(kuu_sym, t_i, x_j) \
        - b*sp.diff(kuu_sym, t_i, y_j) \
        - c*sp.diff(kuu_sym, t_i, x_j, x_j) \
        - d*sp.diff(kuu_sym, t_i, y_j, y_j) \
        + sp.diff(kuu_sym, t_i, t_j)
kff_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, a,b,c,d), kff_sym, "numpy")
def kff(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kff_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t, a,b,c,d)
    return k

$k_{fu}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= a\partial_{x_i}k_{uu} + b \partial_{y_i}k_{uu} + c \partial_{x_i, x_i}k_{uu} + d \partial_{y_i, y_i}k_{uu} -  \partial_{t_i}k_{uu}$

In [122]:
kfu_sym = a*sp.diff(kuu_sym, x_i) \
        + b*sp.diff(kuu_sym, y_i) \
        + c*sp.diff(kuu_sym, x_i, x_i) \
        + d*sp.diff(kuu_sym, y_i, y_i) \
        - sp.diff(kuu_sym, t_i)
kfu_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, a,b,c,d), kfu_sym, "numpy")
def kfu(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kfu_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t, a,b,c,d)
    return k

In [123]:
def kuf(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d):
    return kfu(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d).T

#### Step 3: Compute NLML (with Cholesky decomposition)

Implementing the covariance matrix K and its inverse

In [124]:
def K(sigma, l_x, l_y, l_t, a,b,c,d, s):
    K_mat = np.block([
        [kuu(x, y, t, sigma, l_x, l_y, l_t) + s*np.eye(n), kuf(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d)],
        [kfu(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d), kff(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d) + s*np.eye(n)]
    ])
    return K_mat

In [125]:
def K_inv_and_det(sigma, l_x, l_y, l_t, a,b,c,d, s):
    
    K_inv = np.zeros((2*n, 2*n))
    log_sum = 0
    
    # Use Cholesky, if possible. Otherwise use SVD.
        
    try:
        L = np.linalg.cholesky(K(sigma, l_x, l_y, l_t, a,b,c,d, s))
        L_inv = solve_triangular(L, np.identity(2*n), lower=True) # Slight performance boost over np.linalg.inv
        K_inv = (L_inv.T).dot(L_inv)

        for i in range(2*n):
            log_sum = log_sum + np.log(np.abs(L[i,i]))
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(K(sigma, l_x, l_y, l_t, a,b,c,d, s))
        K_inv = (vt.T).dot(np.linalg.inv(np.diag(s_mat))).dot(u.T)  

        # Calculating the log of the determinant of K
        # Singular values are always positive.
        for i in range(s_mat.size):
            log_sum = log_sum + np.log(s_mat[i])
     
    return K_inv, log_sum

Implementing normalized negative log-likelihood function

In [126]:
def nlml(params):

    
    # Exponentiation to enable unconstrained optimization
    sigma_exp = np.exp(params[0]) 
    l_x_exp = np.exp(params[1])
    l_y_exp = np.exp(params[2]) 
    l_t_exp = np.exp(params[3])
    # a = params[4]
    y_con = np.concatenate((y_u, y_f))
    
    A,b = K_inv_and_det(sigma_exp, l_x_exp, l_y_exp, l_t_exp, params[4],params[5],params[6],params[7], s)
        
    val = b + y_con @ A @ y_con
    return val

#### Step 4: Optimize hyperparameters

**1. Nelder-Mead**

In [127]:
def callbackf(params):
    print(params)

In [128]:
def minimize_restarts(x,y,y_u,y_f,n=10): 
    all_results = []
    for it in range(0,n):
        all_results.append(opt.minimize(nlml, np.random.rand(8), callback = callbackf, method="Nelder-Mead", 
                                        options={'maxfev':5000, 'fatol':0.001, 'xatol':0.001}))
        print('Restart',n,'\n')
    return min(all_results, key = lambda x: x.fun)

In [129]:
t0 = time.time()
m = minimize_restarts(x, y, y_u, y_f, 5)
t_Nelder = time.time() - t0
print(m)                                               

[0.89227081 0.66093472 0.85685387 0.45934203 0.39985206 0.86910291
 0.61670118 0.5627786 ]
[0.89227081 0.66093472 0.85685387 0.45934203 0.39985206 0.86910291
 0.61670118 0.5627786 ]
[0.89227081 0.66093472 0.85685387 0.45934203 0.39985206 0.86910291
 0.61670118 0.5627786 ]
[0.89227081 0.66093472 0.85685387 0.45934203 0.39985206 0.86910291
 0.61670118 0.5627786 ]
[0.89227081 0.66093472 0.85685387 0.45934203 0.39985206 0.86910291
 0.61670118 0.5627786 ]
[0.89227081 0.66093472 0.85685387 0.45934203 0.39985206 0.86910291
 0.61670118 0.5627786 ]
[0.94475878 0.64635728 0.8206295  0.45664129 0.3767649  0.92022803
 0.62744049 0.55374855]
[0.94475878 0.64635728 0.8206295  0.45664129 0.3767649  0.92022803
 0.62744049 0.55374855]
[0.89028091 0.64462295 0.80592815 0.46077655 0.41452206 0.9324807
 0.64331741 0.50231408]
[0.89028091 0.64462295 0.80592815 0.46077655 0.41452206 0.9324807
 0.64331741 0.50231408]
[0.89028091 0.64462295 0.80592815 0.46077655 0.41452206 0.9324807
 0.64331741 0.50231408]
[0

[ 2.58631798 -0.49809479 -1.06746349  0.60262921  0.81529589  2.41435985
  0.70592079  0.41873123]
[ 2.58631798 -0.49809479 -1.06746349  0.60262921  0.81529589  2.41435985
  0.70592079  0.41873123]
[ 2.58631798 -0.49809479 -1.06746349  0.60262921  0.81529589  2.41435985
  0.70592079  0.41873123]
[ 2.70339691 -0.48931119 -0.91870461  0.57833153  0.8963407   2.25691973
  0.53420714  0.42268926]
[ 2.69581253 -0.51388256 -0.9859394   0.5916771   0.85441686  2.281751
  0.62629102  0.3324944 ]
[ 3.19043599 -0.58411627 -1.16018894  0.60840187  0.90260769  2.3755253
  0.44162038  0.34192273]
[ 3.19043599 -0.58411627 -1.16018894  0.60840187  0.90260769  2.3755253
  0.44162038  0.34192273]
[ 3.19043599 -0.58411627 -1.16018894  0.60840187  0.90260769  2.3755253
  0.44162038  0.34192273]
[ 3.19043599 -0.58411627 -1.16018894  0.60840187  0.90260769  2.3755253
  0.44162038  0.34192273]
[ 3.19043599 -0.58411627 -1.16018894  0.60840187  0.90260769  2.3755253
  0.44162038  0.34192273]
[ 3.19043599 -0.5

[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.3621159  -5.37808518 -1.6773615   0.67575977  3.03770646  1.80965991
  0.48434879  0.60818096]
[ 4.362115

[ 5.10167496 -5.17490493 -2.04500304  0.6353824   3.05441482  2.01223065
  0.2433358   0.56144379]
[ 5.10167496 -5.17490493 -2.04500304  0.6353824   3.05441482  2.01223065
  0.2433358   0.56144379]
[ 5.10167496 -5.17490493 -2.04500304  0.6353824   3.05441482  2.01223065
  0.2433358   0.56144379]
[ 5.10167496 -5.17490493 -2.04500304  0.6353824   3.05441482  2.01223065
  0.2433358   0.56144379]
[ 5.05292849 -5.1753775  -2.03151226  0.64068562  3.04678623  2.01120089
  0.2592181   0.56746523]
[ 5.05292849 -5.1753775  -2.03151226  0.64068562  3.04678623  2.01120089
  0.2592181   0.56746523]
[ 5.07606723 -5.16588737 -2.04663086  0.63974451  3.04508351  2.02173304
  0.2542251   0.56404369]
[ 5.06153222 -5.17590281 -2.03547764  0.63868404  3.04879948  2.01186621
  0.25804693  0.56408367]
[ 5.06153222 -5.17590281 -2.03547764  0.63868404  3.04879948  2.01186621
  0.25804693  0.56408367]
[ 5.05545925 -5.16788734 -2.02656353  0.63903074  3.04466651  2.0074064
  0.25709192  0.56395181]
[ 5.0554592

[ 5.77750396 -5.05735493 -1.80233814  0.64835591  2.76828654  2.00150687
  0.35510219  0.57891323]
[ 5.68350259 -5.05185666 -1.76164793  0.64897002  2.77117142  1.97096103
  0.36490423  0.57132742]
[ 5.68350259 -5.05185666 -1.76164793  0.64897002  2.77117142  1.97096103
  0.36490423  0.57132742]
[ 5.73811052 -5.01446385 -1.69390024  0.65105252  2.70051958  1.96506731
  0.41374476  0.57237481]
[ 5.73811052 -5.01446385 -1.69390024  0.65105252  2.70051958  1.96506731
  0.41374476  0.57237481]
[ 6.00776328 -4.91259539 -1.63403604  0.65806668  2.56356454  1.99911557
  0.46283396  0.56612665]
[ 6.00776328 -4.91259539 -1.63403604  0.65806668  2.56356454  1.99911557
  0.46283396  0.56612665]
[ 6.35194418 -4.88612179 -1.53712994  0.6552282   2.46129822  1.97957416
  0.49632963  0.5620807 ]
[ 6.35194418 -4.88612179 -1.53712994  0.6552282   2.46129822  1.97957416
  0.49632963  0.5620807 ]
[ 6.35194418 -4.88612179 -1.53712994  0.6552282   2.46129822  1.97957416
  0.49632963  0.5620807 ]
[ 6.351944

...

[ 2.21180013 -3.94246306 -4.26167334 -3.33540184  7.57532186 -1.65114898
  6.34908575  4.30178717]
[ 2.21180013 -3.94246306 -4.26167334 -3.33540184  7.57532186 -1.65114898
  6.34908575  4.30178717]
[ 2.21180013 -3.94246306 -4.26167334 -3.33540184  7.57532186 -1.65114898
  6.34908575  4.30178717]
[ 2.21180013 -3.94246306 -4.26167334 -3.33540184  7.57532186 -1.65114898
  6.34908575  4.30178717]
[ 2.22358947 -3.94158714 -4.26022647 -3.35512019  7.65096585 -1.71183242
  6.28925971  4.34910692]
[ 2.22358947 -3.94158714 -4.26022647 -3.35512019  7.65096585 -1.71183242
  6.28925971  4.34910692]
[ 2.22358947 -3.94158714 -4.26022647 -3.35512019  7.65096585 -1.71183242
  6.28925971  4.34910692]
[ 2.22358947 -3.94158714 -4.26022647 -3.35512019  7.65096585 -1.71183242
  6.28925971  4.34910692]
[ 2.22358947 -3.94158714 -4.26022647 -3.35512019  7.65096585 -1.71183242
  6.28925971  4.34910692]
[ 2.22358947 -3.94158714 -4.26022647 -3.35512019  7.65096585 -1.71183242
  6.28925971  4.34910692]
[ 2.223589

[ 2.19040809 -3.98811351 -4.23616292 -3.34125617  8.68559827 -2.34152107
  4.86487394  4.77232744]
[ 2.19040809 -3.98811351 -4.23616292 -3.34125617  8.68559827 -2.34152107
  4.86487394  4.77232744]
[ 2.19040809 -3.98811351 -4.23616292 -3.34125617  8.68559827 -2.34152107
  4.86487394  4.77232744]
[ 2.19040809 -3.98811351 -4.23616292 -3.34125617  8.68559827 -2.34152107
  4.86487394  4.77232744]
[ 2.16986663 -3.97186007 -4.2582822  -3.32049001  8.68528677 -2.31005454
  4.85655214  4.73688756]
[ 2.16986663 -3.97186007 -4.2582822  -3.32049001  8.68528677 -2.31005454
  4.85655214  4.73688756]
[ 2.16986663 -3.97186007 -4.2582822  -3.32049001  8.68528677 -2.31005454
  4.85655214  4.73688756]
[ 2.16986663 -3.97186007 -4.2582822  -3.32049001  8.68528677 -2.31005454
  4.85655214  4.73688756]
[ 2.16986663 -3.97186007 -4.2582822  -3.32049001  8.68528677 -2.31005454
  4.85655214  4.73688756]
[ 2.16986663 -3.97186007 -4.2582822  -3.32049001  8.68528677 -2.31005454
  4.85655214  4.73688756]
[ 2.195128

[ 2.18286281 -3.95854792 -4.24453753 -3.28639393  8.62486521 -2.32169963
  4.80945899  4.71232411]
[ 2.17217934 -3.95662412 -4.24392579 -3.31088672  8.65182207 -2.29701196
  4.83306415  4.73037123]
[ 2.17217934 -3.95662412 -4.24392579 -3.31088672  8.65182207 -2.29701196
  4.83306415  4.73037123]
[ 2.17217934 -3.95662412 -4.24392579 -3.31088672  8.65182207 -2.29701196
  4.83306415  4.73037123]
[ 2.17217934 -3.95662412 -4.24392579 -3.31088672  8.65182207 -2.29701196
  4.83306415  4.73037123]
[ 2.17217934 -3.95662412 -4.24392579 -3.31088672  8.65182207 -2.29701196
  4.83306415  4.73037123]
[ 2.17217934 -3.95662412 -4.24392579 -3.31088672  8.65182207 -2.29701196
  4.83306415  4.73037123]
[ 2.18200369 -3.96203303 -4.24095792 -3.28592672  8.62187724 -2.3144179
  4.79997325  4.71545883]
[ 2.1699964  -3.95920366 -4.24230531 -3.31728501  8.67676558 -2.30214325
  4.8107135   4.74503193]
[ 2.1699964  -3.95920366 -4.24230531 -3.31728501  8.67676558 -2.30214325
  4.8107135   4.74503193]
[ 2.1670915

[ 2.18246102 -3.95710966 -4.24488223 -3.28571226  8.36370234 -1.94770387
  4.81223626  4.72008983]
[ 2.18246102 -3.95710966 -4.24488223 -3.28571226  8.36370234 -1.94770387
  4.81223626  4.72008983]
[ 2.18246102 -3.95710966 -4.24488223 -3.28571226  8.36370234 -1.94770387
  4.81223626  4.72008983]
[ 2.18246102 -3.95710966 -4.24488223 -3.28571226  8.36370234 -1.94770387
  4.81223626  4.72008983]
[ 2.18246102 -3.95710966 -4.24488223 -3.28571226  8.36370234 -1.94770387
  4.81223626  4.72008983]
[ 2.1934888  -3.95183627 -4.24954038 -3.24357225  8.15569241 -1.72717286
  4.75304711  4.6967083 ]
[ 2.17912003 -3.94874715 -4.2456118  -3.31075814  8.24484155 -1.7280408
  4.83396225  4.74646283]
[ 2.17912003 -3.94874715 -4.2456118  -3.31075814  8.24484155 -1.7280408
  4.83396225  4.74646283]
[ 2.17912003 -3.94874715 -4.2456118  -3.31075814  8.24484155 -1.7280408
  4.83396225  4.74646283]
[ 2.19921012 -3.95262087 -4.24563153 -3.26369966  8.19798577 -1.77521547
  4.77529326  4.71595544]
[ 2.19921012 

[ 2.21857325 -3.95279311 -4.24103043 -3.23055951  8.02552156 -1.6637351
  4.79815447  4.67932364]
[ 2.21878499 -3.95268836 -4.24126301 -3.22948889  8.02308609 -1.66243858
  4.79753675  4.67830255]
[ 2.21878499 -3.95268836 -4.24126301 -3.22948889  8.02308609 -1.66243858
  4.79753675  4.67830255]
[ 2.2188617  -3.9526509  -4.24115172 -3.22924989  8.0193107  -1.65786717
  4.79755792  4.67814843]
[ 2.2188617  -3.9526509  -4.24115172 -3.22924989  8.0193107  -1.65786717
  4.79755792  4.67814843]
[ 2.21916973 -3.95273943 -4.24108782 -3.22910643  8.01870597 -1.65850663
  4.7982764   4.67788212]
[ 2.21916973 -3.95273943 -4.24108782 -3.22910643  8.01870597 -1.65850663
  4.7982764   4.67788212]
[ 2.21916973 -3.95273943 -4.24108782 -3.22910643  8.01870597 -1.65850663
  4.7982764   4.67788212]
[ 2.21916973 -3.95273943 -4.24108782 -3.22910643  8.01870597 -1.65850663
  4.7982764   4.67788212]
[ 2.21916973 -3.95273943 -4.24108782 -3.22910643  8.01870597 -1.65850663
  4.7982764   4.67788212]
[ 2.2191697

In [130]:
t_Nelder

14604.7439057827

In [131]:
print('The inferred parameters are:')
print('a = ', m.x[4])
print('b = ', m.x[5])
print('c = ', m.x[6])
print('d = ', m.x[7])

The inferred parameters are:
a =  2.017709813544886
b =  1.996843143758807
c =  0.5273923301013722
d =  0.5261195431197652


In [132]:
nlml(m.x)

-618.1283754841504